In [ ]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np
from keras.preprocessing import image
import time
from sklearn.preprocessing import StandardScaler

In [ ]:
labels = pd.read_csv("Datasets/cartoon_set/labels.csv", index_col=0, sep='\t')

In [ ]:
labels.head()

In [ ]:
images_dir_cartoon="Datasets/cartoon_set/img"
basedir_cartoon = "Datasets/cartoon_set"
labels_filename = "labels.csv"

In [ ]:
def read_img():
    all_imgs = []
    all_labels = []
    
    image_paths_cartoon = [os.path.join(images_dir_cartoon, l) for l in os.listdir(images_dir_cartoon)]
    labels_file = open(os.path.join(basedir_cartoon, labels_filename), 'r')
    lines = labels_file.readlines()
    
    face_shape_labels = {line.split('\t')[-1].split('\n')[0] : int(line.split('\t')[2]) for line in lines[1:]}
    
    if os.path.isdir(images_dir_cartoon):
        
        for img_path in image_paths_cartoon:

            file_name = img_path.split('.')[0].split('/')[-1]
#             file_name = img_path.split('.')[1].split('/')[-1]
            
            img = image.img_to_array(image.load_img(img_path,
                                                    target_size=(100,100),
                                                    interpolation='bicubic'))
            all_imgs.append(img)
            all_labels.append(face_shape_labels[file_name+'.png'])
    
    return np.asarray(all_imgs, np.float32), np.asarray(all_labels, np.int32)

In [ ]:
data, label = read_img()

In [ ]:
print(data.shape)
print(label.shape)

In [ ]:
num_example = data.shape[0]
arr = np.arange(num_example)
np.random.shuffle(arr)
data_s = data[arr]
label_s = label[arr]

In [ ]:
ratio = 0.8
s = np.int(num_example * ratio)
x_train = data_s[:s]
y_train = label_s[:s]
x_val = data_s[s:]
y_val = label_s[s:]

In [ ]:
x_train = x_train/255.
x_val = x_val/255.

In [ ]:
x = tf.placeholder(tf.float32, shape=[None, 100, 100, 3], name='x')
y_ = tf.placeholder(tf.int32, shape=[None,], name='y_')

In [ ]:
def inference(input_tensor, train, regularizer):
    with tf.variable_scope('layer1-conv1'):
        conv1_weights = tf.get_variable("weight", 
                                        [5,5,3,32], 
                                        initializer=tf.truncated_normal_initializer(stddev=0.1))
        conv1_biases = tf.get_variable("bias", [32], initializer=tf.constant_initializer(0.0))
        conv1 = tf.nn.conv2d(input_tensor, conv1_weights, strides=[1,1,1,1], padding='SAME')
        relu1 = tf.nn.relu(tf.nn.bias_add(conv1, conv1_biases))
    
    with tf.name_scope("layer2-pool1"):
        pool1 = tf.nn.max_pool(relu1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')
    
    with tf.variable_scope('layer3-conv2'):
        conv2_weights = tf.get_variable("weight", 
                                        [5,5,32,64], 
                                        initializer=tf.truncated_normal_initializer(stddev=0.1))
        conv2_biases = tf.get_variable("bias", [64], initializer=tf.constant_initializer(0.0))
        conv2 = tf.nn.conv2d(pool1, conv2_weights, strides=[1,1,1,1], padding='SAME')
        relu2 = tf.nn.relu(tf.nn.bias_add(conv2, conv2_biases))
    
    with tf.name_scope("layer4-pool2"):
        pool2 = tf.nn.max_pool(relu2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')
    
    with tf.variable_scope('layer5-conv3'):
        conv3_weights = tf.get_variable("weight", 
                                        [3,3,64,128], 
                                        initializer=tf.truncated_normal_initializer(stddev=0.1))
        conv3_biases = tf.get_variable("bias", [128], initializer=tf.constant_initializer(0.0))
        conv3 = tf.nn.conv2d(pool2, conv3_weights, strides=[1,1,1,1], padding='SAME')
        relu3 = tf.nn.relu(tf.nn.bias_add(conv3, conv3_biases))
    
    with tf.name_scope("layer6-pool3"):
        pool3 = tf.nn.max_pool(relu3, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')
    
    with tf.variable_scope('layer7-conv4'):
        conv4_weights = tf.get_variable("weight", 
                                        [3,3,128,128], 
                                        initializer=tf.truncated_normal_initializer(stddev=0.1))
        conv4_biases = tf.get_variable("bias", [128], initializer=tf.constant_initializer(0.0))
        conv4 = tf.nn.conv2d(pool3, conv4_weights, strides=[1,1,1,1], padding='SAME')
        relu4 = tf.nn.relu(tf.nn.bias_add(conv4, conv4_biases))
    
    with tf.name_scope("layer8-pool4"):
        pool4 = tf.nn.max_pool(relu4, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')
        nodes = 6*6*128
        reshaped = tf.reshape(pool4, [-1,nodes])
    
    with tf.variable_scope('layer9-fc1'):
        fc1_weights = tf.get_variable("weight", 
                                      [nodes,1024], 
                                      initializer=tf.truncated_normal_initializer(stddev=0.1))
        if regularizer != None: 
            tf.add_to_collection('losses', regularizer(fc1_weights))
            
        fc1_biases = tf.get_variable("bias", [1024], initializer=tf.constant_initializer(0.1))
        
        fc1 = tf.nn.relu(tf.matmul(reshaped, fc1_weights) + fc1_biases)
        if train:
            fc1 = tf.nn.dropout(fc1, 0.5)
    
    with tf.variable_scope('layer10-fc2'):
        fc2_weights = tf.get_variable("weight", 
                                      [1024,512], 
                                      initializer=tf.truncated_normal_initializer(stddev=0.1))
        if regularizer != None: 
            tf.add_to_collection('losses', regularizer(fc2_weights))
            
        fc2_biases = tf.get_variable("bias", [512], initializer=tf.constant_initializer(0.1))
        
        fc2 = tf.nn.relu(tf.matmul(fc1, fc2_weights) + fc2_biases)
        if train:
            fc2 = tf.nn.dropout(fc2, 0.5)
    
    with tf.variable_scope('layer11-fc3'):
        fc3_weights = tf.get_variable("weight", 
                                      [512,5], 
                                      initializer=tf.truncated_normal_initializer(stddev=0.1))
        if regularizer != None: 
            tf.add_to_collection('losses', regularizer(fc3_weights))
            
        fc3_biases = tf.get_variable("bias", [5], initializer=tf.constant_initializer(0.1))
        
#         logit = tf.nn.softmax(tf.matmul(fc2, fc3_weights) + fc3_biases)
        logit = tf.matmul(fc2, fc3_weights) + fc3_biases
        
    return logit

In [ ]:
regularizer = tf.contrib.layers.l2_regularizer(0.0001)
logits = inference(x, False, regularizer)

In [ ]:
loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y_)
train_op = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss)
correct_prediction = tf.equal(tf.cast(tf.argmax(logits, 1), tf.int32), y_)
acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
def minibatches(inputs=None, targets=None, batch_size=None, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batch_size + 1, batch_size):
        if shuffle:
            excerpt = indices[start_idx: start_idx + batch_size]
        else:
            excerpt = slice(start_idx, start_idx + batch_size)
        yield inputs[excerpt], targets[excerpt]

In [ ]:
n_epoch = 10
batch_size = 50
# saver = tf.train.Saver()
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for epoch in range(n_epoch):
    start_time = time.time()
    print("==========epoch %d=========="%epoch)
    
    train_loss, train_acc, n_batch = 0, 0, 0
    for x_train_a, y_train_a in minibatches(x_train, y_train, batch_size, shuffle=True):
        _, err, ac = sess.run([train_op, loss, acc], feed_dict={x: x_train_a, y_: y_train_a})
        train_loss += err
        train_acc += ac
        n_batch += 1
    print("train loss: %f" % (np.sum(train_loss) / n_batch))
    print("train acc: %f" % (np.sum(train_acc) / n_batch))
    
    val_loss, val_acc, n_batch = 0, 0, 0
    for x_val_a, y_val_a in minibatches(x_val, y_val, batch_size, shuffle=False):
        err, ac = sess.run([loss, acc], feed_dict={x: x_val_a, y_: y_val_a})
        val_loss += err
        val_acc += ac
        n_batch += 1
    print("validation loss: %f" % (np.sum(val_loss) / n_batch))
    print("validation acc: %f" % (np.sum(val_acc) / n_batch))
# saver.save(sess, model_path)
sess.close()